# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

# План выполнения проекта:

1. Загрузка и подготовка данных.
2. Обучение моделей с помощью BERT и TF-IDF.
3. Итоговый вывод

# Описание данных:

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# Подключение библиотек:

In [1]:
!pip install transformers
!pip install catboost

In [2]:
import warnings; warnings.filterwarnings("ignore", category=Warning)
import transformers
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import xgboost as xgb
import torch
import time
 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
 
from sklearn.feature_extraction.text import TfidfVectorizer 
 
from transformers import DistilBertModel, DistilBertConfig
 
from tqdm import notebook
 
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# 1. Загрузка и подготовка данных.

In [3]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', engine="python", error_bad_lines=False)

In [4]:
display(df.head(5))

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
display(df.info())
print(df['toxic'].value_counts(normalize=True))
print('\033[1m' + 'Количество дубликатов в датасете -' + '\033[0m', df.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


None

0    0.898321
1    0.101679
Name: toxic, dtype: float64
Количество дубликатов в датасете - 0


Наблюдаем что в тексте нашего датасета дубликаты отсутствуют, и наблюдаются различные лишние символы - решим эти две проблемы следующим образом:
1. Сразу же подготовим наши признаки и таргет, разделим их на обучающую и тестовую выборку, и после этого выделим одну выборку из 1000 сэмплов для BERT.
2. Очистим текст в датасете от лишних символов, стоп-слов и лемматизируем его (только для датасета под метод TF-IDF). Выполним эти действия с помощью отдельных функций.

In [6]:
random_state = 11111

x_par = df['text']
y_tar = df['toxic']

x_train, x_test, y_train, y_test = train_test_split(x_par, y_tar, test_size = 0.2, random_state=random_state)

print('\033[1m'+'Обучающая выборка'+'\033[0m')
display(y_train.value_counts(normalize=True))
print('')
print('\033[1m'+'Тестовая выборка'+'\033[0m')
display(y_test.value_counts(normalize=True))

Обучающая выборка


0    0.89785
1    0.10215
Name: toxic, dtype: float64


Тестовая выборка


0    0.900204
1    0.099796
Name: toxic, dtype: float64

In [7]:
x_train_bert = x_train.sample(750, replace=True, random_state=random_state)
y_train_bert = y_train.loc[x_train_bert.index]

x_test_bert = x_test.sample(250, replace=True, random_state=random_state)
y_test_bert = y_test.loc[x_test_bert.index]

Классы сбалансированы, данные в порядке, теперь можем переходить к очистке текста текста:

In [8]:
def clean(text):
    
    text = text.lower()    
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    
    return text

x_train = x_train.apply(clean)
x_train_bert = x_train_bert.apply(clean)
x_test = x_test.apply(clean)
x_test_bert = x_test_bert.apply(clean)

Текст очищен от лишних символов, теперь необходимо избавиться от стоп-слов.

In [9]:
stop_words = set(nltk_stopwords.words('english'))
def remove_stopwords(text):
    
    clean_text = [w for w in text.split() if not w in stop_words]
    return clean_text

x_train = x_train.apply(lambda x: remove_stopwords(x))
x_test = x_test.apply(lambda x: remove_stopwords(x))

Теперь лемматизируем датасет для TF-IDF.

In [10]:
%%time
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    clean_text = [lemmatizer.lemmatize(word) for word in text]
    return clean_text

x_train = x_train.apply(lambda x: lemmatize(x))
x_test = x_test.apply(lambda x: lemmatize(x))

CPU times: user 26.9 s, sys: 288 ms, total: 27.2 s
Wall time: 27.2 s


Данные были успешно рассмотрены, очищены от лишних символов, стоп-слов и лемматизированы. Теперь можем переходить к обучению моделей.


# 2. Обучение и анализ моделей.

## 2.1. Модель DistilBert.

Для работы с BERT напишем функцию которая создаст эмбединги для наших признаков. 

In [11]:
tokenizer = transformers.DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def bert_embeddings(features):
    config = DistilBertConfig.from_pretrained('distilbert-base-uncased')
    model = DistilBertModel.from_pretrained('distilbert-base-uncased', config=config)

    tokenized = features.apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))
  
    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)
  
    padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
    attention_mask = np.where(padded != 0, 1, 0)

    batch_size = 50
    embeddings = []
    for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
       
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
       
        embeddings.append(batch_embeddings[0][:,0,:].numpy())
    embeddings_ready = np.concatenate(embeddings)
    return embeddings_ready

Функция готова к применению, создадим эмбединги для обучающей и тестовой выборки

In [12]:
%%time
x_train_bert = bert_embeddings(x_train_bert)
x_test_bert = bert_embeddings(x_test_bert)

print('Обучающая выборка:')
display(x_train_bert.shape)
print('')
print('Тестовая выборка:')
display(x_test_bert.shape)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Обучающая выборка:


(750, 768)


Тестовая выборка:


(250, 768)

CPU times: user 12min 51s, sys: 6.49 s, total: 12min 58s
Wall time: 12min 54s


Данные подготовлены к обучению, теперь напишем функцию которая на входе будет принимать модель, набор гиперпараметров и наименование модели на русском языке. Внутри модель начнет отсчет времени, подготовит кроссвалидацию на основе KFold, определит набор оптимальных гиперпараметров для определенной модели. После этого модель с использованием оптимальных гиперпараметров обучится на обучающей выборке, предскажет значения на обучающей и тестовой выборке, рассчитает значения F1 Score, и сведет результаты в таблицу.

In [13]:
f1_results = pd.DataFrame({'Модель' : [], 'Train F1 Score' : [], 'Test F1 Score' : [], 'Затрачиваемое время, сек.' : []})
tfidf_f1_results = pd.DataFrame({'Модель' : [], 'Train F1 Score' : [], 'Test F1 Score' : [], 'Затрачиваемое время, сек.' : []})
def model_testing(model_name, parametrs, output_model_name, purpose, 
                  features_train, features_test, target_train, target_test):
    
    global f1_results, tfidf_f1_results
    
    start_timer = time.time()

    cv = KFold(n_splits=3, random_state = random_state)
        
    model = model_name()
        
    grid = GridSearchCV(model, parametrs, cv = cv, scoring = 'f1')
    
    grid.fit(features_train, target_train)
    test_model = model_name(**grid.best_params_)
    
    test_model.fit(features_train, target_train)
    model_train_predictions = test_model.predict(features_train)
    model_test_predictions = test_model.predict(features_test)
    train_f1_score = f1_score(target_train, model_train_predictions)
    test_f1_score = f1_score(target_test, model_test_predictions)
    if purpose == 'BERT':
        f1_results = f1_results.append({'Модель' : output_model_name, 
                                      'Train F1 Score' : train_f1_score,
                                      'Test F1 Score' : test_f1_score,
                                      'Затрачиваемое время, сек.' : round(time.time() - start_timer)
                                      }, ignore_index = True)
    elif purpose == 'TF-IDF':
        tfidf_f1_results = tfidf_f1_results.append({'Модель' : output_model_name, 
                                                    'Train F1 Score' : train_f1_score,
                                                    'Test F1 Score' : test_f1_score,
                                                    'Затрачиваемое время, сек.' : round(time.time() - start_timer)
                                                    }, ignore_index = True)
    print('\033[1m' +'Модель -', output_model_name + '\033[0m')
    print('\033[1m' + 'Затрачиваемое время на выполнение -', round(time.time() - start_timer), 'сек. \033[0m')
    print('Оптимальные гиперпараметры:')
    print(grid.best_params_)
    print('')
    print('Train F1 score -', train_f1_score)
    print('Test F1 score -', test_f1_score)

Функция подготовлена к использованию, начнем с логистической регрессии

### 2.1.1. Логистическая регрессия.

In [14]:
parametrs = {'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
             'intercept_scaling': [1.5, 1.65, 1.75, 1.85, 1.95, 2],
             'C': [0.25, 0.5, 0.75, 1]}
model_testing(LogisticRegression, parametrs, 'Логистическая регрессия', 'BERT', 
              x_train_bert, x_test_bert, y_train_bert, y_test_bert)

Модель - Логистическая регрессия
Затрачиваемое время на выполнение - 109 сек. 
Оптимальные гиперпараметры:
{'C': 1, 'intercept_scaling': 1.5, 'solver': 'saga'}

Train F1 score - 0.9325153374233128
Test F1 score - 0.7741935483870969


Видим что с использованием BERT логистическая регрессия с кроссвалидацией и автоподбором параметров показывает значение F1 score на тестовой выборке в 0.85, что является достаточно хорошим результатом при низких временных затратах. Теперь рассмотрим модель "Дерево решений"

### 2.1.2. Ridge Classifier

In [15]:
parametrs = {'alpha': range(1, 5),
             'fit_intercept': [True],
             'normalize':[False],
             'copy_X':[True],
             'solver':['lsqr', 'sparse_cg', 'sag', 'saga'],
             'random_state': [random_state]}
model_testing(RidgeClassifier, parametrs, 'RidgeClassifier', 'BERT', 
              x_train_bert, x_test_bert, y_train_bert, y_test_bert)

Модель - RidgeClassifier
Затрачиваемое время на выполнение - 23 сек. 
Оптимальные гиперпараметры:
{'alpha': 2, 'copy_X': True, 'fit_intercept': True, 'normalize': False, 'random_state': 11111, 'solver': 'lsqr'}

Train F1 score - 0.951219512195122
Test F1 score - 0.6909090909090908


Ridge Classifier также проходит установленный порог в 0.75 F1 Score на тестовой выборке, однако показывает более слабый результат по сравнению с логистической регрессией - F1-мера составила 0.78 на тестовой выборке. Переходим к модели случайного леса.

### 2.1.3. SVM

In [16]:
parametrs = {'kernel': ['rbf'],
             'C': [5, 6, 6.5, 7, 7.5, 8],
             'class_weight': [None, 'balanced'],
             'gamma': [0.05, 0.0057, 0.0055, 0.0053, 0.0051]}
model_testing(SVC, parametrs, 'SVM', 'BERT', 
              x_train_bert, x_test_bert, y_train_bert, y_test_bert)

Модель - SVM
Затрачиваемое время на выполнение - 26 сек. 
Оптимальные гиперпараметры:
{'C': 7, 'class_weight': None, 'gamma': 0.05, 'kernel': 'rbf'}

Train F1 score - 1.0
Test F1 score - 0.7241379310344829


SVM, аналогично, проходит порог в 0.75 F1-меры, результаты являются достаточно оптимальными при временных затратах как у логистической регрессии. Рассмотрим модель CatBoost Classifier.

### 2.1.4. CatBoost Classifier

In [17]:
parametrs = {'max_depth' : [4], 
             'loss_function' : ['Logloss'], 
             'eval_metric' : ['F1'], 
             'logging_level' : ['Silent'], 
             'iterations' : [90],
             'auto_class_weights': ['Balanced']}
model_testing(CatBoostClassifier, parametrs, 'CatBoost Classifier', 'BERT', 
              x_train_bert, x_test_bert, y_train_bert, y_test_bert)

Модель - CatBoost Classifier
Затрачиваемое время на выполнение - 37 сек. 
Оптимальные гиперпараметры:
{'auto_class_weights': 'Balanced', 'eval_metric': 'F1', 'iterations': 90, 'logging_level': 'Silent', 'loss_function': 'Logloss', 'max_depth': 4}

Train F1 score - 1.0
Test F1 score - 0.7096774193548386


Модель CatBoost Classifier показывает худшие результаты по времени, однако достаточно хорошие результаты по качеству, в случае если запас времени не ограничен - возможно кэтбуст сможет выдать наилучшие результаты по качеству. Теперь переходим к модели LGBM Classifier

### 2.1.5. LGBM Classifier

In [18]:
parametrs = {'max_depth': range(1, 30, 5),
            'learning_rate' : [0.01, 0.05, 0.1],
             'is_unbalance': ['True']}
model_testing(LGBMClassifier, parametrs, 'LGBM Classifier', 'BERT', 
              x_train_bert, x_test_bert, y_train_bert, y_test_bert)

Модель - LGBM Classifier
Затрачиваемое время на выполнение - 265 сек. 
Оптимальные гиперпараметры:
{'is_unbalance': 'True', 'learning_rate': 0.1, 'max_depth': 1}

Train F1 score - 0.8911917098445596
Test F1 score - 0.6865671641791045


Модель LGBM Classifier показывает средние результаты по времени, чуть дольше чем логистическая регрессия, однако по качеству не сильно уступает кэтбусту и регрессии. И переходим к последней рассматриваемой модели - XGBoost Classifier

### 2.1.6. XGBoost Classifier

In [19]:
parametrs = {'n_estimators': range(5, 50, 10), 
             'max_depth': range(20, 50, 5), 
             'silent' : [True],
             'scale_pos_weight': [1]}
model_testing(xgb.XGBClassifier, parametrs, 'XGBoost Classifier', 'BERT', 
              x_train_bert, x_test_bert, y_train_bert, y_test_bert)

Модель - XGBoost Classifier
Затрачиваемое время на выполнение - 100 сек. 
Оптимальные гиперпараметры:
{'max_depth': 20, 'n_estimators': 25, 'scale_pos_weight': 1, 'silent': True}

Train F1 score - 1.0
Test F1 score - 0.5882352941176471


XGBoost показывает аналогичным образом как с LGBM Classifier достаточно средние результаты и сильно проигрывает в качестве другим моделям.

In [20]:
display(f1_results)

,Модель,Train F1 Score,Test F1 Score,"Затрачиваемое время, сек."
0,Логистическая регрессия,0.932515,0.774194,109.0
1,RidgeClassifier,0.951220,0.690909,23.0
2,SVM,1.000000,0.724138,26.0
3,CatBoost Classifier,1.000000,0.709677,37.0
4,LGBM Classifier,0.891192,0.686567,265.0
5,XGBoost Classifier,1.000000,0.588235,100.0


По итогам использования BERT все используемые модели прошли порог в 0.75 F1-меры на тестовой выборке, однако для данного метода потребуются большие вычислительные мощности и большой запас времени так как разметка эмбеддингов занимает очень много времени.

## 2.2. Использование TF-IDF.

В первую очередь необходимо подготовить признаки и целевой признак, подготовить корпуса и TfidfVectorizer

In [21]:
x_train = x_train.astype('U').values
x_test = x_test.astype('U').values

count_tf_idf = TfidfVectorizer(stop_words=stop_words)
tf_idf = count_tf_idf.fit(x_train)

x_train = tf_idf.transform(x_train)
x_test = tf_idf.transform(x_test)

Данные подготовлены к обучению, будем использовать аналогичную функцию для BERT, а в качестве моделей возьмем три модели, показавших наилучшие результаты на BERT - т.е. логистическую регрессию, LGBM Classifier и XGBoost Classifier. CatBoost не будем использовать так как его выполнение занимает достаточно много времени

### 2.2.1. Логистическая регрессия

In [22]:
parametrs = {'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
             'intercept_scaling': [0.5, 1.0],
             'C': [0.5, 1, 2]}
model_testing(LogisticRegression, parametrs, 'Логистическая регрессия', 'TF-IDF', 
              x_train, x_test, y_train, y_test)

Модель - Логистическая регрессия
Затрачиваемое время на выполнение - 341 сек. 
Оптимальные гиперпараметры:
{'C': 2, 'intercept_scaling': 0.5, 'solver': 'saga'}

Train F1 score - 0.8035247752734358
Test F1 score - 0.7574640619240692


### 2.2.2. LGBM Classifier

In [23]:
parametrs = {'max_depth': range(3, 10, 3),
            'learning_rate' : [0.05, 0.1]}
model_testing(LGBMClassifier, parametrs, 'LGBM Classifier', 'TF-IDF', 
              x_train, x_test, y_train, y_test)

Модель - LGBM Classifier
Затрачиваемое время на выполнение - 487 сек. 
Оптимальные гиперпараметры:
{'learning_rate': 0.1, 'max_depth': 9}

Train F1 score - 0.7108917598103257
Test F1 score - 0.6770645870825834


### 2.2.3. XGBoost Classifier

In [24]:
parametrs = {'n_estimators': range(5, 15, 5), 
             'silent' : [True]}
model_testing(xgb.XGBClassifier, parametrs, 'XGBoost Classifier', 'TF-IDF', 
              x_train, x_test, y_train, y_test)

Модель - XGBoost Classifier
Затрачиваемое время на выполнение - 41 сек. 
Оптимальные гиперпараметры:
{'n_estimators': 5, 'silent': True}

Train F1 score - 0.42309994015559543
Test F1 score - 0.437137330754352


In [25]:
display(tfidf_f1_results)
display(f1_results)

,Модель,Train F1 Score,Test F1 Score,"Затрачиваемое время, сек."
0,Логистическая регрессия,0.803525,0.757464,341.0
1,LGBM Classifier,0.710892,0.677065,487.0
2,XGBoost Classifier,0.423100,0.437137,41.0


,Модель,Train F1 Score,Test F1 Score,"Затрачиваемое время, сек."
0,Логистическая регрессия,0.932515,0.774194,109.0
1,RidgeClassifier,0.951220,0.690909,23.0
2,SVM,1.000000,0.724138,26.0
3,CatBoost Classifier,1.000000,0.709677,37.0
4,LGBM Classifier,0.891192,0.686567,265.0
5,XGBoost Classifier,1.000000,0.588235,100.0


# 3. Итоговый вывод.

По итогу данного проекта были выполнены следующие шаги:
- Проверен датасет на наличие дубликатов, пропусков. Датасет был очищен от лишних символов/стопслов и лемматизирован отдельно для BERT и TF-IDF.
- Датасет был подготовлен для BERT и TF-IDF: для БЕРТ была выполнена токенизация и эмбеддинг, для TF-IDF перевели признаки в корпус, разделили на выборки и с помощью TfidfVectorizer подготовили данные к обучению на моделях
- Провели обучение и тестирование моделей: на BERT лучшими моделями по метрике f1-меры на тестовой выборке оказались логистическая регрессия (0.7741), SVM (0.7241), и Ridge Classifier (0.6909). На TF-IDF лучший результат показала модель логистической регрессии (0.7574) при cредних временных затратах. **Порог по метрике (0.75 F1 Score) с помощью BERT и TF-IDF смогла пройти только логистическая регрессия**